<a href="https://colab.research.google.com/github/AV-016/Research-on-LSTM/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras_tuner

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

# Load your data (example for one ticker)
def load_data(ticker, sequence_length=60):
    df = yf.download(ticker, period='5y')
    df = df[['Close', 'High', 'Low', 'Volume']]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i - sequence_length:i])
        y.append(scaled_data[i, 0])  # predicting 'Close'

    return np.array(X), np.array(y), scaler

# Model builder function
def build_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Int('lstm_units', min_value=32, max_value=256, step=32),
        return_sequences=False,
        input_shape=input_shape
    ))
    model.add(Dense(hp.Int('dense_units', min_value=16, max_value=128, step=16)))
    model.add(Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='mean_squared_error'
    )
    return model

# Tuner function
def run_tuning(X_train, y_train, X_val, y_val):
    tuner = kt.Hyperband(
        build_model,
        objective='val_loss',
        max_epochs=20,
        factor=3,
        directory='tuning_logs',
        project_name='lstm_tune'
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=3)

    tuner.search(
        X_train, y_train,
        epochs=20,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1
    )

    best_model = tuner.get_best_models(num_models=1)[0]
    return best_model, tuner

# Run the tuning
ticker = 'AAPL'
X, y, scaler = load_data(ticker)
input_shape = (X.shape[1], X.shape[2])

# Split the data
split_index = int(len(X) * 0.8)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

best_model, tuner = run_tuning(X_train, y_train, X_val, y_val)

# Evaluate model
predictions = best_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
mape = mean_absolute_percentage_error(y_val, predictions)

print(f'📊 {ticker} RMSE: {rmse:.4f}')
print(f'📊 {ticker} MAPE: {mape:.2%}')


Trial 30 Complete [00h 00m 54s]
val_loss: 0.0012066987110301852

Best val_loss So Far: 0.0006280053639784455
Total elapsed time: 00h 15m 05s
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step
📊 AAPL RMSE: 0.0251
📊 AAPL MAPE: 2.18%


In [4]:
tickers = ['MSFT', 'GOOGL', 'AMZN','AAPL']
results = []

for ticker in tickers:
    print(f"\n📈 Processing {ticker}...")

    df = yf.download(ticker, start="2020-01-01", end="2024-01-01")
    df = df[['Close', 'High', 'Low', 'Volume']]

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length - 7):
            X.append(data[i:i + seq_length])
            y.append(data[i + seq_length:i + seq_length + 7, 0])
        return np.array(X), np.array(y)

    SEQ_LEN = 60
    X, y = create_sequences(scaled, SEQ_LEN)

    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    # 👇 Use the best hyperparameters (from tuner)
    best_units = 64
    best_dense = 64
    best_batch_size = 32
    best_epochs = 50

    model = Sequential([
        LSTM(best_units, return_sequences=True, input_shape=(SEQ_LEN, X.shape[2])),
        LSTM(best_units),
        Dense(best_dense, activation='relu'),
        Dense(7)  # 7-day prediction
    ])

    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, validation_split=0.1, epochs=best_epochs,
              batch_size=best_batch_size, verbose=0)

    # Predictions
    preds_7d = model.predict(X_test)
    preds_1d = preds_7d[:, 0]  # 1-day ahead
    true_1d = y_test[:, 0]

    # Inverse scale for RMSE & MAPE
    close_index = df.columns.get_loc("Close")
    y_test_rescaled = scaler.inverse_transform(np.concatenate(
        [y_test[:, [0]], np.zeros((y_test.shape[0], 3))], axis=1))[:, 0]
    preds_1d_rescaled = scaler.inverse_transform(np.concatenate(
        [preds_1d.reshape(-1, 1), np.zeros((len(preds_1d), 3))], axis=1))[:, 0]

    rmse_1d = np.sqrt(np.mean((preds_1d_rescaled - y_test_rescaled) ** 2))
    mape_1d = np.mean(np.abs((preds_1d_rescaled - y_test_rescaled) / y_test_rescaled)) * 100

    y_test_7d = y_test[:, :7].mean(axis=1)
    preds_7d_mean = preds_7d.mean(axis=1)

    y_test_7d_rescaled = scaler.inverse_transform(np.concatenate(
        [y_test_7d.reshape(-1, 1), np.zeros((len(y_test_7d), 3))], axis=1))[:, 0]
    preds_7d_rescaled = scaler.inverse_transform(np.concatenate(
        [preds_7d_mean.reshape(-1, 1), np.zeros((len(preds_7d_mean), 3))], axis=1))[:, 0]

    rmse_7d = np.sqrt(np.mean((preds_7d_rescaled - y_test_7d_rescaled) ** 2))
    mape_7d = np.mean(np.abs((preds_7d_rescaled - y_test_7d_rescaled) / y_test_7d_rescaled)) * 100

    results.append([ticker, rmse_1d, mape_1d, rmse_7d, mape_7d])

    print(f"📊 {ticker} RMSE (1-day): {rmse_1d:.4f}")
    print(f"📊 {ticker} MAPE (1-day): {mape_1d:.2f}%")
    print(f"📊 {ticker} RMSE (7-day): {rmse_7d:.4f}")
    print(f"📊 {ticker} MAPE (7-day): {mape_7d:.2f}%")


[*********************100%***********************]  1 of 1 completed


📈 Processing MSFT...



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step


[*********************100%***********************]  1 of 1 completed

...

📊 GOOGL RMSE (1-day): 3.1762
📊 GOOGL MAPE (1-day): 1.92%
📊 GOOGL RMSE (7-day): 4.0916
📊 GOOGL MAPE (7-day): 2.40%

📈 Processing AMZN...



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step
📊 AMZN RMSE (1-day): 3.8582
📊 AMZN MAPE (1-day): 2.34%
📊 AMZN RMSE (7-day): 6.2705
📊 AMZN MAPE (7-day): 3.86%

📈 Processing AAPL...


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step
📊 AAPL RMSE (1-day): 4.5006
📊 AAPL MAPE (1-day): 2.12%
📊 AAPL RMSE (7-day): 6.2087
📊 AAPL MAPE (7-day): 2.97%
